#  하이브리드 검색
---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

---

## RAG 검색기

1. **의미론적 검색 (Semantic Search)**
    - Vector Store를 기반으로 한 검색 방식으로, 텍스트의 의미적 유사성을 고려하여 검색을 수행함
    - 임베딩 벡터 간의 유사도를 계산하여 의미적으로 관련성이 높은 문서를 찾아내는 특징이 있음
    - 동의어나 문맥적 의미를 파악할 수 있어 자연어 질의에 효과적임

1. **키워드 검색 (Keyword Search)**
    - BM25와 같은 전통적인 검색 알고리즘을 사용하여 키워드 매칭을 기반으로 검색을 수행함
    - 정확한 단어나 구문 매칭에 강점이 있으며, 계산 효율성이 높은 특징을 가짐
    - 직접적인 키워드 일치를 찾는 데 유용하나, 의미적 유사성을 파악하는 데는 한계가 있음

1. **하이브리드 검색 (Hybrid Search)**
    - 키워드 기반 검색과 의미론적 검색을 결합한 방식으로, EnsembleRetriever를 통해 구현됨
    - 두 검색 방식의 장점을 활용하여 더 정확하고 포괄적인 검색 결과를 제공함
    - 정확한 키워드 매칭과 의미적 연관성을 모두 고려하여 검색 성능을 향상시키는 특징이 있음

### 1) **Semantic Search** (의미론적 검색) 

- **의미론적 검색**은 텍스트의 **벡터 표현**을 활용해 의미적 유사성 기반 검색 수행
- **Vector Store**에 저장된 임베딩 벡터 간 **유사도 계산**으로 관련 문서 검색
- 검색어와 문서 간의 **문맥적 의미**와 **동의어 관계**를 효과적으로 파악
- **자연어 질의**에 강점을 보이며 기존 키워드 검색의 한계를 보완
- 전통적인 검색 방식과 달리 **의미 기반 매칭**으로 더 정확하고 포괄적인 검색 결과 제공

`(1) 벡터 저장소 초기화`
- cosine distance 기준으로 인덱싱 

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

korean_txt_files = glob(os.path.join('data', '*_KR.md')) 
korean_data = load_text_files(korean_txt_files)

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",    # TikToken 인코더 이름
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
    chunk_size=300,            # 문서 분할 크기
    chunk_overlap=50,          # 문서 분할 중첩  
    is_separator_regex=True,      # 구분자가 정규식인지 여부
    keep_separator=True,          # 구분자 유지 여부
)

korean_chunks = text_splitter.split_documents(korean_data)

print("한국어 청크 수:", len(korean_chunks))

In [ ]:
for i, doc in enumerate(korean_chunks):
    print(f"[{i}]", doc.page_content)
    print("="*200)

In [ ]:
from langchain_core.documents import Document
# Document 객체에 메타데이터 추가

korean_docs = []

for chunk in korean_chunks:
    doc = Document(page_content=chunk.page_content, metadata=chunk.metadata)
    doc.metadata['company'] = '테슬라' if '테슬라' in doc.metadata['source'] else '리비안'
    doc.metadata['language'] = 'ko'
    doc.page_content = f"<Document>\n{doc.page_content}\n</Document>\n<Source>이 문서는 미국 전기차 회사인 '{doc.metadata['company']}'에 대한 문서입니다.</Source>"   
    korean_docs.append(doc)

print("한국어 문서 수:", len(korean_docs))
print("="*200)
print(korean_docs[0].metadata)
print("-"*200)
print(korean_docs[0].page_content)
print("="*200)
print(korean_docs[-1].metadata)
print("-"*200)
print(korean_docs[-1].page_content)


In [ ]:
# korean_docs 파일을 jsonlines 파일로 저장
def save_jsonlines(docs, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for doc in docs:
            f.write(json.dumps(doc.model_dump_json(), ensure_ascii=False) + '\n')

save_jsonlines(korean_docs, 'data/korean_docs_final.jsonl')

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
chroma_db = Chroma.from_documents(
    documents=korean_docs,
    embedding=embeddings,    
    collection_name="db_korean_cosine_metadata", 
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

In [ ]:
# 저장된 문서 수 확인
print("Chroma DB에 저장된 문서 수:", chroma_db._collection.count())

`(2) 벡터 저장소 로드`
- 미리 인덱싱해서 저장해 둔 저장소를 가져와서 사용
- 이때 기존에 사용한 임베딩 모델을 초기화 필요

In [ ]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

In [ ]:
# 저장된 문서 수 확인
print("Chroma DB에 저장된 문서 수:", chroma_db._collection.count())

`(3) Semantic Search 실행`
- 벡터 저장소 검색기 객체 활용
- 임베딩 벡터 간의 유사도를 기반으로 문서 검색

In [ ]:
# 검색기 지정하여 테스트 
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

for doc in retrieved_docs:
    print(doc.page_content)
    print("="*200)

### 2) **Keyword Search** (키워드 검색) 

- **키워드 검색**은 **BM25** 등 전통적 알고리즘 기반의 단어 매칭 방식
- 정확한 **단어/구문 매칭**에 강점이 있으며 **계산 효율성**이 우수함
- **직접적인 키워드** 검색에는 효과적이나 의미적 연관성 파악에는 제한적
- 구현이 단순하고 **처리 속도가 빠르다**는 장점이 있음
- 정확한 키워드 매칭이 필요한 경우에 적합하나 **의미론적 검색의 보완**이 필요함

`(1) BM25 검색기 생성`

- BM25: TF-IDF (Term Frequency-Inverse Document Frequency)의 확장된 버전
- `rank_bm25` 설치

In [ ]:
# 벡터 저장소에 저정한 문서 객체를 로드하여 확인
chroma_db.get().keys()

In [ ]:
# BM25 검색기 생성을 위해 문서 객체를 로드
documents = chroma_db.get()["documents"]
metadatas = chroma_db.get()["metadatas"]

# Document 객체로 변환
from langchain_core.documents import Document
docs = [Document(page_content=content, metadata=meta) for content, meta in zip(documents, metadatas)]

print("문서의 수:" , len(docs))
print("=" * 200)
for doc in docs[:3]:
    print(doc.page_content)
    print("-" * 200)
    print(doc.metadata)
    print("=" * 200)

In [ ]:
from langchain_community.retrievers import BM25Retriever

# BM25 검색기 생성
bm25_retriever = BM25Retriever.from_documents(docs)

# BM25 검색기를 사용하여 검색
query = "리비안은 언제 사업을 시작했나요?"

retrieved_docs = bm25_retriever.invoke(query)

for doc in retrieved_docs:
    print(doc.page_content)
    print("="*200)

In [ ]:
# BM25 점수를 확인 - "리비안은" 라는 단어가 쿼리에 포함되어 있지 않아 검색 결과가 없음
query = "리비안은 언제 사업을 시작했나요?"
tokenized_query = query.split()
print(tokenized_query)
print("="*200)

# 문서의 BM25 점수 확인
doc_scores = bm25_retriever.vectorizer.get_scores(tokenized_query)

# 문서의 BM25 점수를 내림차순으로 정렬
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)

# 상위 5개 문서의 인덱스와 점수를 출력
for idx, score in doc_scores_sorted[:5]:
    print(f"[{idx}] {docs[idx].page_content}")
    print(f"BM25 Score: {score}")
    print("-"*200)

In [ ]:
# 같은 의미를 갖는 쿼리로 변경하여 다시 검색 
query = "리비안이 설립된 연도는?"

retrieved_docs = bm25_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
# BM25 점수를 확인 - "설립된" 이라는 단어가 쿼리에 포함되어 있어 검색 결과가 있음
query = "리비안이 설립된 연도는?"
tokenized_query = query.split()
print(tokenized_query)
print("="*200)

# 문서의 BM25 점수 확인
doc_scores = bm25_retriever.vectorizer.get_scores(tokenized_query)

# 문서의 BM25 점수를 내림차순으로 정렬
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)

# 상위 5개 문서의 인덱스와 점수를 출력
for idx, score in doc_scores_sorted[:5]:
    print(f"[{idx}] {docs[idx].page_content}")
    print(f"BM25 Score: {score}")
    print("-"*200)

`(2) kiwi 한국어 토크나이저`
- `kiwipiepy` 설치 필요 (pip install kiwipiepy / uv add kiwipiepy)

In [ ]:
# 한국어 토크나이저를 사용하여 문장을 토큰화
from kiwipiepy import Kiwi

kiwi_model = Kiwi()

print(kiwi_model.analyze("리비안은 언제 사업을 시작했나요?"))
print("="*200)
print(kiwi_model.analyze("리비안이 설립된 연도는?"))

In [ ]:
print(kiwi_model.tokenize("테슬라는 언제 설립되었나요?"))
print("="*200)
print(kiwi_model.tokenize("테슬라가 설립된 연도는?"))

In [ ]:
# 단어를 추가 
kiwi_model.add_user_word('리비안', 'NNP')  # NNP: 고유명사

print(kiwi_model.analyze("리비안은 언제 사업을 시작했나요?"))
print("="*200)
print(kiwi_model.analyze("리비안이 설립된 연도는?"))

In [ ]:
# 한국어 토크나이저를 사용하여 문장을 토큰화하는 함수를 정의 

def bm25_kiwi_process_func(text):
    """
    BM25Retriever에서 사용할 전처리 함수
    한국어 토크나이저를 사용하여 문장을 토큰화 (Kiwi 사용)
    :param text: 토큰화할 문장
    """
    # 한국어 토크나이저를 사용하여 문장을 토큰화
    return [t.form for t in kiwi_model.tokenize(text)]


# BM25Retriever 객체 생성
bm25_kiwi_retriever = BM25Retriever.from_documents(
    documents=docs,
    preprocess_func=bm25_kiwi_process_func, # 전처리 함수 지정
    )

In [ ]:
# 이전에 사용한 검색어를 입력하여 문서를 검색
query = "리비안이 설립된 연도는?"

retrieved_docs = bm25_kiwi_retriever.invoke(query)

for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")
    print("="*200)

In [ ]:
# BM25 점수를 확인
query = "리비안이 설립된 연도는?"

tokenized_query = [t.form for t in kiwi_model.tokenize(query)]
print(tokenized_query)
print("="*200)

# 문서의 BM25 점수 확인
doc_scores = bm25_kiwi_retriever.vectorizer.get_scores(tokenized_query)

# 문서의 BM25 점수를 내림차순으로 정렬
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)

# 상위 5개 문서의 인덱스와 점수를 출력
for idx, score in doc_scores_sorted[:5]:
    print(f"[{idx}] {docs[idx].page_content}")
    print(f"BM25 Score: {score}")
    print("-"*200)

### 3) **Hybrid Search** (하이브리드 검색) 

- **하이브리드 검색**은 **키워드 검색**과 **의미론적 검색**을 **EnsembleRetriever**로 통합
- 정확한 **키워드 매칭**과 **의미적 유사성**을 동시에 고려하여 검색 수행
- 두 검색 방식의 **장점을 결합**하여 더 포괄적이고 정확한 결과 도출
- 검색 성능 향상을 위해 각 방식의 **가중치 조정**이 가능함
- 키워드와 의미 기반 검색의 **시너지 효과**로 더 향상된 검색 성능 실현 가능

In [ ]:
from langchain.retrievers import EnsembleRetriever

# 앙상블 검색기 생성
ensemble_retrievers = [chroma_k_retriever, bm25_kiwi_retriever]

ensemble_retriever = EnsembleRetriever(
    retrievers=ensemble_retrievers, 
    weights=[0.5, 0.5]          # 각 검색기의 가중치
)

# 검색기를 사용하여 검색
query = "리비안이 설립된 연도는?"

retrieved_docs = ensemble_retriever.invoke(query)

for doc in retrieved_docs:
    print(doc.page_content)
    print("="*200)

---

## 검색 성능 평가

### 1) **테스트 데이터** 

- 합성된 데이터는 **품질 검증**과 **수동 수정** 과정을 거쳐 정제
- 테스트용 데이터는 **다양한 유형의 질문**과 **답변 패턴**을 포함해야 함
- 신뢰할 수 있는 검색 성능 평가를 위해 **고품질 테스트 데이터** 확보가 중요함

In [ ]:
# 기존에 생성해 둔 테스트셋 로드
# 테스트셋 로드
import pandas as pd
df_qa_test = pd.read_excel("data/testset.xlsx")

print(f"테스트셋: {df_qa_test.shape[0]}개 문서")
df_qa_test.head(2)

### 2) **Information Retrieval 평가지표**

- K-RAG 패키지 사용 (pip install krag)
- Hit Rate, MRR, mAP@k, NDCG@k 계산

`(1) 테스트 데이터셋의 컨텍스트를 문서 객체로 변환`

In [ ]:
# 테스트 데이터셋의 특정 행에 있는 컨텍스트 데이터를 Document 객체 리스트로 변환
from langchain_core.documents import Document

context_docs = []
for i, row in df_qa_test.iterrows():
    row_docs = []
    for doc in eval(row['reference_contexts']):
        row_docs.append(Document(page_content=doc))

    context_docs.append(row_docs)


print(f"컨텍스트 문서: {len(context_docs)}개 문서")
print("="*200)
print(context_docs[0])

In [ ]:
# korean_docs 문서가 저장되어 있는 jsonlines 파일을 로드
korean_docs = []
with open('data/korean_docs_final.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        doc = json.loads(line)
        korean_docs.append(Document.model_validate_json(doc))
        
print(f"한국어 문서: {len(korean_docs)}개 문서")

In [ ]:
korean_docs[0]

`(2) 키워드 검색 (Kiwi 토크나이저 + BM25 검색기)`

- krag 파키지 설지 : https://pypi.org/project/krag/
- pip install krag 또는 uv pip install krag 

In [ ]:
from krag.tokenizers import KiwiTokenizer
from krag.retrievers import KiWiBM25RetrieverWithScore

# 한국어 토크나이저 생성
kiwi_tokenizer = KiwiTokenizer(model_type='knlm', typos='basic')

# '리비안' 단어 추가
kiwi_tokenizer.kiwi.add_user_word('리비안', 'NNP')  # NNP: 고유명사     

# 토큰화 테스트
print(kiwi_tokenizer.tokenize("리비안은 언제 사업을 시작했나요?"))

In [ ]:
# BM25 검색기 초기화 (k=3)
retriever_bm25_kiwi = KiWiBM25RetrieverWithScore(
    documents=korean_docs, 
    kiwi_tokenizer=kiwi_tokenizer, 
    k=3, 
) 

# 검색기 테스트
query = "리비안이 설립된 연도는?"
retrieved_docs = retriever_bm25_kiwi.invoke(query)
for doc in retrieved_docs:
    print(doc.page_content)
    print("="*200)

In [ ]:
# BM25 점수를 확인
query = "리비안이 설립된 연도는?"

tokenized_query = [t.form for t in kiwi_tokenizer.kiwi.tokenize(query)]
print(tokenized_query)
print("="*200)

# 문서의 BM25 점수 확인
doc_scores = retriever_bm25_kiwi.bm25_retriever.vectorizer.get_scores(tokenized_query)

# 문서의 BM25 점수를 내림차순으로 정렬
doc_scores_sorted = sorted(enumerate(doc_scores), key=lambda x: x[1], reverse=True)

# 상위 5개 문서의 인덱스와 점수를 출력
for idx, score in doc_scores_sorted[:5]:
    print(f"[{idx}] {korean_docs[idx].page_content}")
    print(f"BM25 Score: {score}")
    print("-"*200)

In [ ]:
# BM25 검색기를 사용하여 문서 검색
question = df_qa_test['user_input'].iloc[0]
print("질문:", question)
print("="*200)
context = df_qa_test['reference_contexts'].iloc[0]
print("관련 문서:", context)
print("="*200)

# BM25 검색
retrieved_docs = retriever_bm25_kiwi.invoke(question)

# 검색 결과 출력 
for doc in retrieved_docs:
    print(f"BM25 점수: {doc.metadata["bm25_score"]:.2f}")    
    print(f"\n{doc.page_content}\n[출처: {doc.metadata['company']}]")
    print("-"*200)

In [ ]:
# 전체 테스트 데이터셋에 대하여 평가지표 계산
from langchain_core.retrievers import BaseRetriever
from krag.evaluators import RougeOfflineRetrievalEvaluators

def evaluate_qa_test(df_qa_test: pd.DataFrame, retriever: BaseRetriever, k=2) -> dict:
    """
    테스트 데이터셋에 대한 검색 결과 평가
    """

    context_docs = []
    retrieved_docs = []

    df_test = df_qa_test.copy()
    
    for idx, _ in df_test.iterrows():
        question = df_test['user_input'].iloc[idx]
        context_doc = [Document(page_content=doc) for doc in eval(df_test['reference_contexts'].iloc[idx])]
        context_docs.append(context_doc)
        retrieved_doc = retriever.invoke(question)  
        retrieved_docs.append(retrieved_doc)  


    # 평가자 인스턴스 생성
    evaluator = RougeOfflineRetrievalEvaluators(
        actual_docs=context_docs,
        predicted_docs=retrieved_docs, 
        match_method='rouge2',
        threshold=0.8,
    )


    # 평가지표 계산
    precision = evaluator.calculate_precision(k=k)['micro_precision']
    recall = evaluator.calculate_recall(k=k)['micro_recall']
    f1_score = evaluator.calculate_f1_score(k=k)['micro_f1']
    hit_rate = evaluator.calculate_hit_rate(k=k)['hit_rate']
    mrr = evaluator.calculate_mrr(k=k)['mrr']
    map_score = evaluator.calculate_map(k=k)['map']
    ndcg = evaluator.calculate_ndcg(k=k)['ndcg']

    print(f"K={k}")
    print("-"*200)
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1_score:.3f}")
    print(f"Hit Rate: {hit_rate:.3f}")
    print(f"MRR: {mrr:.3f}")
    print(f"MAP: {map_score:.3f}")
    print(f"NDCG: {ndcg:.3f}")
    print("="*200)
    print()

    result = {
        'k': k,
        'precision': precision,
        'recall': recall,
        'f1_score': f1_score,
        'hit_rate': hit_rate,
        'mrr': mrr,
        'map': map_score,
        'ndcg': ndcg,
        
    }

    return pd.Series(result)

In [ ]:
# 평가 (k=1)
retriever_bm25_kiwi.k = 1
result_bm25_k1 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=1)

In [ ]:
# 평가 (k=2)
retriever_bm25_kiwi.k = 2
result_bm25_k2 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=2)

In [ ]:
# 평가 (k=3)
retriever_bm25_kiwi.k = 3
result_bm25_k3 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=3)

In [ ]:
# 평가 (k=4)
retriever_bm25_kiwi.k = 4
result_bm25_k4 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=4)

In [ ]:
# 평가 (k=5)
retriever_bm25_kiwi.k = 5
result_bm25_k5 = evaluate_qa_test(df_qa_test, retriever_bm25_kiwi, k=5)

`(3) 시맨틱 검색 (Chroma 벡터저장소 검색기)`

In [ ]:
# Chroma 검색기 초기화
retriever_chroma_db = chroma_db.as_retriever(
    search_kwargs={"k": 5},
)

In [ ]:
# 평가 (k=1)
result_chroma_db_k1 = evaluate_qa_test(df_qa_test, retriever_chroma_db, k=1)

In [ ]:
# 평가 (k=2)
result_chroma_db_k2 = evaluate_qa_test(df_qa_test, retriever_chroma_db, k=2)

In [ ]:
# 평가 (k=3)
result_chroma_db_k3 = evaluate_qa_test(df_qa_test, retriever_chroma_db, k=3)

In [ ]:
# 평가 (k=4)
result_chroma_db_k4 = evaluate_qa_test(df_qa_test, retriever_chroma_db, k=4)

In [ ]:
# 평가 (k=5)
result_chroma_db_k5 = evaluate_qa_test(df_qa_test, retriever_chroma_db, k=5)

`(4) 하이브리드 검색 (EnsembleRetriever 사용)`

In [ ]:
from langchain.retrievers import EnsembleRetriever

retriever_bm25_kiwi.k = 5
ensemble_retrievers = [retriever_chroma_db, retriever_bm25_kiwi]
ensemble_retriever = EnsembleRetriever(
    retrievers=ensemble_retrievers, 
    weights=[0.5, 0.5]
)

# 평가 (k=1)
result_ensemble_k1 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=1)

In [ ]:
# 평가 (k=2)
result_ensemble_k2 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=2)

In [ ]:
# 평가 (k=3)
result_ensemble_k3 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=3)

In [ ]:
# 평가 (k=4)
result_ensemble_k4 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=4)

In [ ]:
# 평가 (k=5)
result_ensemble_k5 = evaluate_qa_test(df_qa_test, ensemble_retriever, k=5)

`(5) 검색 성능 비교`

In [ ]:
# 3가지 검색기의 평가 결과를 하나의 DataFrame으로 결합 (각 검색기별로 k=1, 2, 3, 4, 5에 대한 결과 포함)
import pandas as pd

# 결과를 딕셔너리 리스트로 만들기
results_data = [
    {'retriever': 'BM25-Kiwi', 'k': 1, **result_bm25_k1.to_dict()},
    {'retriever': 'Chroma-DB', 'k': 1, **result_chroma_db_k1.to_dict()},
    {'retriever': 'Ensemble', 'k': 1, **result_ensemble_k1.to_dict()},
    {'retriever': 'BM25-Kiwi', 'k': 2, **result_bm25_k2.to_dict()},
    {'retriever': 'Chroma-DB', 'k': 2, **result_chroma_db_k2.to_dict()},
    {'retriever': 'Ensemble', 'k': 2, **result_ensemble_k2.to_dict()},
    {'retriever': 'BM25-Kiwi', 'k': 3, **result_bm25_k3.to_dict()},
    {'retriever': 'Chroma-DB', 'k': 3, **result_chroma_db_k3.to_dict()},
    {'retriever': 'Ensemble', 'k': 3, **result_ensemble_k3.to_dict()},
    {'retriever': 'BM25-Kiwi', 'k': 4, **result_bm25_k4.to_dict()},
    {'retriever': 'Chroma-DB', 'k': 4, **result_chroma_db_k4.to_dict()},
    {'retriever': 'Ensemble', 'k': 4, **result_ensemble_k4.to_dict()},
    {'retriever': 'BM25-Kiwi', 'k': 5, **result_bm25_k5.to_dict()},
    {'retriever': 'Chroma-DB', 'k': 5, **result_chroma_db_k5.to_dict()},
    {'retriever': 'Ensemble', 'k': 5, **result_ensemble_k5.to_dict()}
]

# 데이터프레임 생성
results = pd.DataFrame(results_data)

# 결과 표시
results


In [ ]:
# 다양한 검색기들의 성능 비교를 위한 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 스타일 설정
sns.set(style="whitegrid")

# 검색기별로 각 평가지표를 시각화
def plot_retriever_performance(results, metric):
    plt.figure(figsize=(12, 6))
    sns.barplot(data=results, x='k', y=metric, hue='retriever')
    plt.title(f'{metric} by Retriever and k')
    plt.xlabel('k')
    plt.ylabel(metric)
    plt.legend(title='Retriever')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Precision 시각화
plot_retriever_performance(results, 'precision')

# Recall 시각화
plot_retriever_performance(results, 'recall')

# F1 Score 시각화
plot_retriever_performance(results, 'f1_score')

# Hit Rate 시각화
plot_retriever_performance(results, 'hit_rate')

# MRR 시각화
plot_retriever_performance(results, 'mrr')

# MAP 시각화
plot_retriever_performance(results, 'map')

# NDCG 시각화
plot_retriever_performance(results, 'ndcg')